In [2]:
# Change the directory path
%cd ../featureSelection/

C:\Users\mikec\Codes\DAADRISE_AbusiveLangProject\featureSelection


In [4]:
import pandas as pd
import h2o
h2o.init()
from h2o.automl import H2OAutoML
from sklearn import metrics

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,2 mins 31 secs
H2O cluster timezone:,Europe/Berlin
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.5
H2O cluster version age:,27 days
H2O cluster name:,H2O_from_python_mikec_akgrqp
H2O cluster total nodes:,1
H2O cluster free memory:,1.741 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [5]:
X_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\UnivariateFeatures.csv'))
y_train = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TrainLabels.csv'))
X_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\UnivariateFeaturesTest.csv'))
y_test = h2o.H2OFrame(pd.read_csv('C:\\Users\\mikec\\Documents\\TestLabels.csv')) 


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
# preparing the train and test data sets
# now convert tweet vecs and labels to a pandas dataframe and back to h2o dataframe
# data = job_titles[valid_job_titles,:].cbind(job_title_vecs[valid_job_titles,:])
train = X_train.cbind(y_train)
test = X_test.cbind(y_test)


In [7]:
# more on data prep
x = train.columns         # x: A list/vector of predictor column names or indexes. 
                          # This argument only needs to be specified if the user wants to exclude columns from the 
                          # set of predictors. If all columns (other than the response) should be used in prediction, 
                          # then this does not need to be set.

y = "labels"              # This argument is the name (or index) of the response column
x.remove(y)

# need to set train and test
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [8]:
# now the AUTO-ML piece comes in
aml = H2OAutoML(max_models=10) #max_models=10 or 20?, seed?
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |█████████████████████████████████████████████████████████| 100%


In [9]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,mean_per_class_error,logloss,rmse,mse
StackedEnsemble_BestOfFamily_AutoML_20190716_142734,0.495895,0.469692,0.382292,0.146147
StackedEnsemble_AllModels_AutoML_20190716_142734,0.499103,0.465409,0.381667,0.145669
GBM_4_AutoML_20190716_142734,0.499409,0.451556,0.380391,0.144698
GBM_2_AutoML_20190716_142734,0.500663,0.446323,0.378108,0.142966
GBM_1_AutoML_20190716_142734,0.503834,0.446871,0.378567,0.143313
GBM_3_AutoML_20190716_142734,0.50688,0.449292,0.379983,0.144387
DeepLearning_1_AutoML_20190716_142734,0.507779,0.478662,0.393869,0.155133
XRT_1_AutoML_20190716_142734,0.511671,0.564981,0.38234,0.146184
DRF_1_AutoML_20190716_142734,0.512341,0.612543,0.384286,0.147676
GBM_5_AutoML_20190716_142734,0.518633,0.445624,0.378923,0.143583


In [10]:
# The leader model is stored here
aml.leader

Model Details
H2OStackedEnsembleEstimator :  Stacked Ensemble
Model Key:  StackedEnsemble_BestOfFamily_AutoML_20190716_142734
No model summary for this model


ModelMetricsMultinomialGLM: stackedensemble
** Reported on train data. **

MSE: 0.12751102495447594
RMSE: 0.3570868591176045

ModelMetricsMultinomialGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.14614699609105922
RMSE: 0.38229176827530464


In [11]:
preds = aml.predict(test)
print(preds)

stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1,p2
1,0.0533817,0.803028,0.14359
1,0.0552645,0.908163,0.0365727
2,0.0437635,0.405957,0.55028
1,0.0621332,0.83863,0.0992363
1,0.195651,0.754768,0.0495803
1,0.0969759,0.849422,0.0536025
1,0.0365563,0.880968,0.0824753
1,0.0361413,0.897004,0.0668552
1,0.0606979,0.735728,0.203574
1,0.087728,0.696559,0.215713


In [12]:
var = preds["predict"].cbind(test[y])
print(var)

predict,labels
1,1
1,1
2,1
1,2
1,1
1,1
1,1
1,1
1,1
1,1


In [13]:
# convert to pandas dataframe
y_test = h2o.as_list(test[y], use_pandas=True)
y_pred = h2o.as_list(preds["predict"])
print(metrics.confusion_matrix(y_test, y_pred))
print(metrics.accuracy_score(y_test, y_pred))

[[   2  249   34]
 [   8 2740 1088]
 [   0  748   83]]
0.5704765751211631
